## Import Packages

In [ ]:
import albumentations
import albumentations.pytorch
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm.notebook import tqdm

import timm
import ttach as tta
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from sklearn.metrics import f1_score
from efficientnet_pytorch import EfficientNet

# from muar.augmentations import BatchRandAugment, MuAugment
from facenet_pytorch import MTCNN
tqdm.pandas()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is using!")

## Config Setting

In [ ]:
## efficientnet-b7
e7_config = {'NUM_EPOCHS' : 30,
             'BATCH_SIZE' : 7,
             'NUM_CLASSES' : 18,
             'LEARNING_RATE' : 3e-4,
             'MODEL' : 'efficientnet-b7',  # 불러올 모델 이름
             'MODEL_NAME' : 'efficientnet-b7',  # 저장할 떄 이름
             'NUM_WORKERS' : 1,
             'LOG_STEPS' : 450,
             'SAVE_PATH' : './epoch/',
             'LOAD_MODEL' : True,  # 학습을 이어서 할 때 True
             'LOAD_MODEL_PATH' : './epoch/e7_weight/epoch_14_e7_0.8227513227513228.pth',  # 이어서 학습할 파일의 경로
             'BETA' : 1.0
         }

## efficientnet-b8
e8_config = {'NUM_EPOCHS' : 30,
             'BATCH_SIZE' : 20,
             'NUM_CLASSES' : 18,
             'LEARNING_RATE' : 3e-4,
             'MODEL' : 'efficientnet-b8',  # 불러올 모델 이름
             'MODEL_NAME' : 'efficientnet-b8',  # 저장할 떄 이름
             'NUM_WORKERS' : 1,
             'LOG_STEPS' : 450,
             'SAVE_PATH' : './epoch/',
             'LOAD_MODEL' : True,  # 학습을 이어서 할 때 True
             'LOAD_MODEL_PATH' : './epoch/efficientnet-b8_epoch_10_0.8402953853416509.pth',  # 이어서 학습할 파일의 경로
             'BETA' : 1.0
         }

## Initiate Wandb

In [ ]:
wandb.init(entity="minibatch28", project="MaskClassification", name="My_Model")

## Augmentation Setting

In [ ]:
# efficientnet-b7 augmentation
e7_train_transform = albumentations.Compose([albumentations.Resize(600,600),
                                             albumentations.RandomRotation(15),
                                             albumentations.HorizontalFlip(p=0.3),
                                             albumentations.OneOf([albumentations.ShiftScaleRotate(rotate_limit=15, p=0.5),
                                                                   albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                                                                   albumentations.HorizontalFlip(p=0.5),
                                                                   albumentations.MotionBlur(p=0.5),
                                                                   albumentations.OpticalDistortion(p=0.5),
                                                                   albumentations.GaussNoise(p=0.5)], p=1),
                                             albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
                                             albumentations.pytorch.transforms.ToTensorV2()])

e8_test_transform = albumentations.Compose([albumentations.Resize(600,600),
                                            albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
                                            albumentations.pytorch.transforms.ToTensorV2()])


# efficientnet-b8 augmentation
e8_train_transform = albumentations.Compose([albumentations.Resize(336,336),
                                             albumentations.OneOf([albumentations.GaussNoise()], p=0.2),
                                             albumentations.OneOf([albumentations.MotionBlur(p=.2),
                                                                   albumentations.MedianBlur(blur_limit=3, p=0.1),
                                                                   albumentations.Blur(blur_limit=3, p=0.1)], p=0.2),
                                             albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
                                             albumentations.OneOf([albumentations.OpticalDistortion(p=0.3),
                                                                albumentations.GridDistortion(p=.1)], p=0.2),
                                             albumentations.OneOf([albumentations.CLAHE(clip_limit=2),
                                                                   albumentations.Sharpen(),
                                                                   albumentations.Emboss(),
                                                                   albumentations.RandomBrightnessContrast()], p=0.3),
                                             albumentations.HueSaturationValue(p=0.3),
                                             albumentations.Cutout(num_holes=4, max_h_size=3, max_w_size=3, fill_value=0, p=0.2),
                                             albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
                                             albumentations.pytorch.transforms.ToTensorV2()])

e8_test_transform = albumentations.Compose([albumentations.Resize(336,336),
                                            albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
                                            albumentations.pytorch.transforms.ToTensorV2()])

## Read DataFrame

In [ ]:
def make_test_full_path(s):
    path = 'input/data/eval/images/'
    return path + s


# 58세까지로 클래스를 잡았을 경우
# efficientnet-b7 사용
e7_train_df = pd.read_csv('./stratified_df/train_df.csv')
e7_valid_df = pd.read_csv('./stratified_df/valid_df.csv')

e7_test_df = pd.read_csv('./input/data/eval/info.csv')
e7_test_df['full_path'] = test_df['ImageID'].progress_apply(make_test_full_path)
e7_submission_df = pd.read_csv('./input/data/eval/info.csv')

# 60세까지로 클래스를 잡았을 경우
# efficientnet-b8 사용
e8_train_df = pd.read_csv('./stratified_df/train_df_0901.csv')
e8_valid_df = pd.read_csv('./stratified_df/valid_df_0901.csv')

e8_test_df = pd.read_csv('./input/data/eval/info.csv')
e8_test_df['full_path'] = test_df['ImageID'].progress_apply(make_test_full_path)
e8_submission_df = pd.read_csv('./input/data/eval/info.csv')

## Dataset & DataLoader

In [ ]:
class E7TrainDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']
        return X, y

    
class E8TrainDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # 얼굴 탐색
            mtcnn = MTCNN()
            boxes, prob = mtcnn.detect(img)
            
            # 얼굴을 찾지 못하는 경우
            if not isinstance(boxes, np.ndarray):
                img = img[110:382, 90:287]
            
            # 얼굴을 찾은 경우
            else:
                xmin, ymin, xmax, ymax = map(int, boxes[0])
                # 표준편차로 얼굴 주변까지 일부 확보
                xmin, ymin, xmax, ymax = abs(xmin)-19, abs(ymin)-35, abs(xmax)+20, abs(ymax)+38
                if xmin <= 0: xmin = 0
                if ymin <= 0: ymin = 0
                img = img[ymin:ymax, xmin:xmax]
            
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']
        return X, y

In [ ]:
class E7TestDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']
        return X
    

class E8TestDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # 얼굴 탐색
            mtcnn = MTCNN()
            boxes, prob = mtcnn.detect(img)
            
            # 얼굴을 찾지 못하는 경우
            if not isinstance(boxes, np.ndarray):
                img = img[110:382, 90:287]
            
            # 얼굴을 찾은 경우
            else:
                xmin, ymin, xmax, ymax = map(int, boxes[0])
                # 표준편차로 얼굴 주변까지 일부 확보
                xmin, ymin, xmax, ymax = abs(xmin)-19, abs(ymin)-35, abs(xmax)+20, abs(ymax)+38
                if xmin <= 0: xmin = 0
                if ymin <= 0: ymin = 0
                img = img[ymin:ymax, xmin:xmax]
            
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']
        return X

In [ ]:
def get_dataset(df, transform, train=True):
    if train:
        dataset = TrainDataset(path=df['full_path'].values,
                               label=df['label'].values,
                               transform=transform)
    else:
        dataset = TestDataset(path=df['full_path'].values,
                              label=df['ans'].values,
                              transform=transform)
    return dataset

def get_loader(dataset, config, shuffle=True):
    loader = DataLoader(dataset, batch_size=config['BATCH_SIZE'], shuffle=shuffle, 
                        num_workers=config['NUM_WORKERS'], pin_memory=True)
    return loader

In [ ]:
# efficientnet-b7
dataset_train = get_dataset(e7_train_df, e7_train_transform, train=True)
dataset_valid = get_dataset(e7_valid_df, e7_test_transform, train=True)

train_dataloader = get_loader(e7_dataset_train, e7_config, shuffle=True)
valid_dataloader = get_loader(e7_dataset_valid, e7_config, shuffle=False)

# efficientnet-b8
dataset_train = get_dataset(e8_train_df, e8_train_transform, train=True)
dataset_valid = get_dataset(e8_valid_df, e8_test_transform, train=True)

train_dataloader = get_loader(e8_dataset_train, e8_config, shuffle=True)
valid_dataloader = get_loader(e8_dataset_valid, e8_config, shuffle=False)

## Modeling

In [ ]:
# efficientnet-b8
class Net(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.net = EfficientNet.from_pretrained(config['MODEL'], num_classes=1024, advprop=True).to(device)
        self.fc = nn.Sequential(nn.Linear(1024,512),
                                nn.LeakyReLU(),
                                nn.Dropout(0.5),
                                nn.Linear(512,config['NUM_CLASSES'])).to(device)
    
    def forward(self, x):
        x = self.net(x)
        x = self.fc(x)
        return x

In [ ]:
def rand_bbox(size, lam):
    W = size[2] 
    H = size[3] 
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)  

    # 패치의 중앙 좌표 값 cx, cy
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # 패치 모서리 좌표 값 
    bbx1 = 0
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = W
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [ ]:
def e7_train(model, config, train_dataloader, device):
    running_loss = 0.0
    log_step_loss = 0.0
    model.train()
    
    for step, (inputs, labels) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        inputs = inputs.to(device).float()
        labels = labels.to(device)
        logits = model(inputs)
        loss = criterion(logits, labels)        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        log_step_loss += loss.item()
        
        if step % config['LOG_STEPS'] == config['LOG_STEPS'] - 1:
            step_loss = log_step_loss / config['LOG_STEPS']
            print(f'Traning Steps : {step + 1} Traning Loss : {step_loss}')
            log_step_loss = 0.0
            
    scheduler.step()
    return running_loss

def e8_train(model, config, train_dataloader, device):
    running_loss = 0.0
    log_step_loss = 0.0
    model.train()
    
    for step, (inputs, labels) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        inputs = inputs.to(device).float()
        labels = labels.to(device)
        
        if config['BETA'] > 0 and np.random.random() > 0.5: # cutmix가 실행될 경우     
            lam = np.random.beta(config['BETA'], config['BETA'])
            rand_index = torch.randperm(inputs.size()[0]).to(device)
            target_a = labels # 원본 이미지 label
            target_b = labels[rand_index] # 패치 이미지 label       
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            logits = model(inputs)
            loss = criterion(logits, target_a) * lam + criterion(logits, target_b) * (1. - lam)
        
        else: # cutmix가 실행되지 않았을 경우
            logits = model(inputs) 
            loss= criterion(logits, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        log_step_loss += loss.item()
        
        if step % config['LOG_STEPS'] == config['LOG_STEPS'] - 1:
            step_loss = log_step_loss / config['LOG_STEPS']
            print(f'Traning Steps : {step + 1} Traning Loss : {step_loss}')
            log_step_loss = 0.0
            
    scheduler.step()
    return running_loss

In [ ]:
def valid(model, valid_dataloader, device):
    gt_list = []
    pred_list = []
    
    model.eval()
    running_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for x, y in tqdm(valid_dataloader):
            x = x.to(device).float()
            y = y.to(device)
            
            logits = model(x)
            loss = criterion(logits, y)
            running_loss += loss.item()
            
            _, pred = torch.max(logits, 1)
            correct += torch.sum(pred == y.data)
            
            for i in y.cpu().numpy():
                gt_list.append(i)
            for j in pred.cpu().numpy():
                pred_list.append(j)
    
    f1 = f1_score(gt_list, pred_list, average='macro')
    acc = correct / dataset_valid.__len__()
    del gt_list, pred_list
    
    print(f'Validation f1_score : {f1}')
    print(f'Validation accuracy : {acc}')
    return f1, acc, running_loss

In [ ]:
def run(model, config, train_dataloader, valid_dataloader, device):
    best_f1 = 0.0
    best_valid_loss = 1e9
    for epoch in range(config['NUM_EPOCHS']):
        print(f'Epoch : {epoch + 1}')
        train_loss = train(model, config, train_dataloader, device)
        f1, acc, valid_loss = valid(model, valid_dataloader, device)
        
        wandb.log({'Train Loss': train_loss,
                   'Validation Loss': valid_loss,
                   'Validation F1': f1,
                   'Validation Acc': acc})
        
        if f1 > best_f1:
            best_f1 = f1
            save_path = config['SAVE_PATH']
            model_name = config['MODEL_NAME']
            torch.save(model.state_dict(), f'{save_path}/{model_name}_epoch_{epoch}_{best_f1}.pth')
        
        elif valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            save_path = config['SAVE_PATH']
            model_name = config['MODEL_NAME']
            torch.save(model.state_dict(), f'{save_path}/{model_name}_epoch_{epoch}_{best_f1}.pth')
        else:
            torch.save(model.state_dict(), f'{save_path}/{model_name}_epoch_{epoch}_0831.pth')
        print('-'*50)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        
        return F.nll_loss(((1 - prob) ** self.gamma) * log_prob,
                           target_tensor,
                          weight=self.weight,
                          reduction=self.reduction)

In [ ]:
e7_model = EfficientNet.from_pretrained(e7_config['MODEL'], num_classes=1024, advprop=True).to(device)
if e7_config['LOAD_MODEL']:
    e7_model.load_state_dict(torch.load(e7_config['LOAD_MODEL_PATH']))
    
e8_model = Net(config)
if e8_config['LOAD_MODEL']:
    e8_model.load_state_dict(torch.load(e8_config['LOAD_MODEL_PATH']))

In [ ]:
# efficientnet-b7
classes = e7_train_df['label'].value_counts().sort_index().values
class_weight = torch.tensor(np.max(classes) / classes).to(device, dtype=torch.float)
e7_criterion = nn.CrossEntropyLoss(weight=class_weight)
e7_optimizer = AdamW(e7_model.parameters(), lr=e7_config['LEARNING_RATE'])
e7_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)

# efficientnet-b8
e8_criterion = FocalLoss()
e8_optimizer = AdamW(e8_model.parameters(), lr=e8_config['LEARNING_RATE'])
e8_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)

In [ ]:
# efficientnet-b8
run(e7_model, e7_config, e7_train_dataloader, e7_valid_dataloader, device)

# efficientnet-b8
run(e8_model, e8_config, e8_train_dataloader, e8_valid_dataloader, device)